In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import coo_matrix, hstack
from sklearn.preprocessing import LabelEncoder

In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)

In [3]:
oe = ordinal_encoder()

In [4]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_pre = pd.read_pickle(ENV.previous_application_cleaned.value)
print('Previous App shape: {}'.format(X_pre.shape))

X_bu_b = pd.read_pickle(ENV.bureau_balance_clean_rnn.value)
print('Bureau Balance shape: {}'.format(X_bu_b.shape))

X_bu_bo = pd.read_pickle(ENV.bureau_balance_clean.value)
print('Bureau Balance shape: {}'.format(X_bu_bo.shape))

X_bu = pd.read_pickle(ENV.bureau_cleaned.value)
print('Bureau shape: {}'.format(X_bu.shape))



X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

X_pos = pd.read_pickle(ENV.POS_CASH_balance_clean.value)
print('POS CASH shape: {}'.format(X_pos.shape))

X_cc = pd.read_pickle(ENV.credit_card_balance_clean.value)
print('Credit Card shape: {}'.format(X_cc.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Previous App shape: (1670214, 37)
Bureau Balance shape: (817395, 10)
Bureau Balance shape: (27299925, 3)
Bureau shape: (1716428, 17)
Installment shape: (13605401, 8)
POS CASH shape: (10001358, 8)
Credit Card shape: (3840312, 23)


In [5]:
X_bu_ori = pd.read_csv(ENV.bureau_ori.value)
print('Bureau shape: {}'.format(X_bu_ori.shape))

Bureau shape: (1716428, 17)


# Bureau

In [6]:
X = X_bu_ori.merge(X_bu_b,how='left',left_on='SK_ID_BUREAU',right_on='SK_ID_BUREAU')

### fillna step 1

In [7]:
for col in ['STATUS0','STATUS1','STATUS2','STATUS3','STATUS4','STATUS5','STATUS6','STATUS7','BUREAU_LENGTH']:
    X[col] = X[col].fillna(0)

### fillna step 2:

##### process AMT_ANUITY

In [8]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'AMT_ANNUITY'
X[col] = X[col].fillna(0)

# option 2: mean
# X[col] = X[col].fillna(X[col].mean())   

##### process AMT_CREDIT_MAX_OVERDUE

In [9]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'AMT_CREDIT_MAX_OVERDUE'
X[col] = X[col].fillna(0)

# option 2: mean
# X[col] = X[col].fillna(X[col].mean())   

##### process DAYS_ENDDATE_FACT

In [10]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'DAYS_ENDDATE_FACT'
# X[col] = X[col].fillna(0)

# option 2: mean
X[col] = X[col].fillna(X[col].mean())   

##### process AMT_CREDIT_SUM_LIMIT

In [11]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'AMT_CREDIT_SUM_LIMIT'
X[col] = X[col].fillna(0)

# option 2: mean
# X[col] = X[col].fillna(X[col].mean())   

##### process AMT_CREDIT_SUM_DEBT       

In [12]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'AMT_CREDIT_SUM_DEBT'
X[col] = X[col].fillna(0)

# option 2: mean
# X[col] = X[col].fillna(X[col].mean())  

##### process DAYS_CREDIT_ENDDATE

In [13]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'DAYS_CREDIT_ENDDATE'
X[col] = X[col].fillna(0)

# option 2: mean
# X[col] = X[col].fillna(X[col].mean())   

# option 3: middle
X[col] = X[col].fillna(-3.3*10**2)  

##### process AMT_CREDIT_SUM

In [14]:
# option 1: 0 fillna
# reason: 50% of non na data is 0
col = 'AMT_CREDIT_SUM'
# X[col] = X[col].fillna(0)

# option 2: mean
X[col] = X[col].fillna(X[col].mean())   

In [15]:
col = 'AMT_CREDIT_SUM'
X[col][X[col].notnull()].describe()

count    1.716428e+06
mean     3.549946e+05
std      1.149807e+06
min      0.000000e+00
25%      5.130000e+04
50%      1.255230e+05
75%      3.150000e+05
max      5.850000e+08
Name: AMT_CREDIT_SUM, dtype: float64

# PROCESS CAT columns

##### process CREDIT_ACTIVE

In [16]:
col = 'CREDIT_ACTIVE'
X[col].value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64

In [17]:
#Option 1: one hot encoding
X['Closed'] = 0
X['Active'] = 0
X['Sold'] = 0
X['Bad_debt'] = 0
X.loc[X[col]=='Closed','Closed'] = 1
X.loc[X[col]=='Active','Active'] = 1
X.loc[X[col]=='Sold','Sold'] = 1
X.loc[X[col]=='Bad debt','Bad_debt'] = 1
X = X.drop(['CREDIT_ACTIVE'],axis=1)

In [18]:
# # Option 2: ordinal encoder
# oe.fit(X[col])
# X[col] = oe.transform(X[col])

##### process CREDIT_CURRENCY

In [19]:
col = 'CREDIT_CURRENCY'
X[col].value_counts()

currency 1    1715020
currency 2       1224
currency 3        174
currency 4         10
Name: CREDIT_CURRENCY, dtype: int64

In [20]:
#Option 1: one hot encoding
X['currency_1'] = 0
X['currency_2'] = 0
X['currency_3'] = 0
X['currency_4'] = 0
X.loc[X[col]=='currency 1','currency_1'] = 1
X.loc[X[col]=='currency 2','currency_2'] = 1
X.loc[X[col]=='currency 3','currency_3'] = 1
X.loc[X[col]=='currency 4','currency_4'] = 1
X = X.drop(['CREDIT_CURRENCY'],axis=1)

In [21]:
# # Option 2: ordinal encoder
# oe.fit(X[col])
# X[col] = oe.transform(X[col])

##### process CREDIT_TYPE

In [22]:
col = 'CREDIT_TYPE'
X[col].value_counts()

Consumer credit                                 1251615
Credit card                                      402195
Car loan                                          27690
Mortgage                                          18391
Microloan                                         12413
Loan for business development                      1975
Another type of loan                               1017
Unknown type of loan                                555
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     27
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: CREDIT_TYPE, dtype: int64

In [23]:
#Option 1: one hot encoding
X['Consumer_credit'] = 0
X['Credit_card'] = 0
X['Car_loan'] = 0
X['Mortgage'] = 0
X['Microloan'] = 0
X['Loan_for_business_development'] = 0
X['Another_type_of_loan'] = 0
X['Unknown_type_of_loan'] = 0
X['Loan_for_working_capital_replenishment'] = 0
X['Cash_loan_(non-earmarked)'] = 0
X['Real_estate_loan'] = 0
X['Loan_for_the_purchase_of_equipment'] = 0
X['Loan_for_purchase_of_shares'] = 0
X['Interbank_credit'] = 0
X['Mobile_operator_loan'] = 0
X.loc[X[col]=='Consumer credit','Consumer_credit'] = 1
X.loc[X[col]=='Credit card','Credit_card'] = 1
X.loc[X[col]=='Car loan','Car_loan'] = 1
X.loc[X[col]=='Mortgage','Mortgage'] = 1
X.loc[X[col]=='Microloan','Microloan'] = 1
X.loc[X[col]=='Loan for business development','Loan_for_business_development'] = 1
X.loc[X[col]=='Another type of loan','Another_type_of_loan'] = 1
X.loc[X[col]=='Unknown type of loan','Unknown_type_of_loan'] = 1
X.loc[X[col]=='Loan for working capital replenishment','Loan_for_working_capital_replenishment'] = 1
X.loc[X[col]=='Cash loan (non-earmarked)','Cash_loan_(non-earmarked)'] = 1
X.loc[X[col]=='Real estate loan','Real_estate_loan'] = 1
X.loc[X[col]=='Loan for the purchase of equipment','Loan_for_the_purchase_of_equipment'] = 1
X.loc[X[col]=='Loan for purchase of shares','Loan_for_purchase_of_shares'] = 1
X.loc[X[col]=='Interbank credit','Interbank_credit'] = 1
X.loc[X[col]=='Mobile operator loan','Mobile_operator_loan'] = 1
X = X.drop(['CREDIT_TYPE'],axis=1)

In [24]:
# # Option 2: ordinal encoder
# oe.fit(X[col])
# X[col] = oe.transform(X[col])

# Aggregate

### Step 1:  get squre:

In [30]:
from itertools import combinations
trans_col = [
             'AMT_ANNUITY',
             'AMT_CREDIT_MAX_OVERDUE',
             'DAYS_ENDDATE_FACT',
             'AMT_CREDIT_SUM_LIMIT',
             'AMT_CREDIT_SUM_DEBT',
             'DAYS_CREDIT_ENDDATE',
             'AMT_CREDIT_SUM',
             'CREDIT_DAY_OVERDUE',
             'CNT_CREDIT_PROLONG',
             'BUREAU_LENGTH',
             'AMT_CREDIT_SUM_OVERDUE',
             'DAYS_CREDIT_UPDATE']
squre_col = []
for col in trans_col:
    col_name = col + '_squre'
    squre_col.append(col_name)
    X[col_name] = X[col] * X[col]
comb_col = []
for col in combinations(trans_col,2):
    col_name = col[0] + '_' + col[1]
    comb_col.append(col_name)
    X[col_name] = X[col[0]] * X[col[1]]

In [31]:
squre_col

['AMT_ANNUITY_squre',
 'AMT_CREDIT_MAX_OVERDUE_squre',
 'DAYS_ENDDATE_FACT_squre',
 'AMT_CREDIT_SUM_LIMIT_squre',
 'AMT_CREDIT_SUM_DEBT_squre',
 'DAYS_CREDIT_ENDDATE_squre',
 'AMT_CREDIT_SUM_squre',
 'CREDIT_DAY_OVERDUE_squre',
 'CNT_CREDIT_PROLONG_squre',
 'BUREAU_LENGTH_squre',
 'AMT_CREDIT_SUM_OVERDUE_squre',
 'DAYS_CREDIT_UPDATE_squre']

In [32]:
comb_col

['AMT_ANNUITY_AMT_CREDIT_MAX_OVERDUE',
 'AMT_ANNUITY_DAYS_ENDDATE_FACT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_LIMIT',
 'AMT_ANNUITY_AMT_CREDIT_SUM_DEBT',
 'AMT_ANNUITY_DAYS_CREDIT_ENDDATE',
 'AMT_ANNUITY_AMT_CREDIT_SUM',
 'AMT_ANNUITY_CREDIT_DAY_OVERDUE',
 'AMT_ANNUITY_CNT_CREDIT_PROLONG',
 'AMT_ANNUITY_BUREAU_LENGTH',
 'AMT_ANNUITY_AMT_CREDIT_SUM_OVERDUE',
 'AMT_ANNUITY_DAYS_CREDIT_UPDATE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_ENDDATE',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM',
 'AMT_CREDIT_MAX_OVERDUE_CREDIT_DAY_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_CNT_CREDIT_PROLONG',
 'AMT_CREDIT_MAX_OVERDUE_BUREAU_LENGTH',
 'AMT_CREDIT_MAX_OVERDUE_AMT_CREDIT_SUM_OVERDUE',
 'AMT_CREDIT_MAX_OVERDUE_DAYS_CREDIT_UPDATE',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_LIMIT',
 'DAYS_ENDDATE_FACT_AMT_CREDIT_SUM_DEBT',
 'DAYS_ENDDATE_FACT_DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT_AMT_CRED

In [33]:
X.to_pickle(ENV.bureau_cleaned_rnnALL.value)

In [26]:
report = scan_nan_portion(X)
report.sort_values(ascending=False)

Mobile_operator_loan                      0.0
AMT_CREDIT_SUM_OVERDUE                    0.0
STATUS6                                   0.0
STATUS5                                   0.0
STATUS4                                   0.0
STATUS3                                   0.0
STATUS2                                   0.0
STATUS1                                   0.0
STATUS0                                   0.0
AMT_ANNUITY                               0.0
DAYS_CREDIT_UPDATE                        0.0
AMT_CREDIT_SUM_LIMIT                      0.0
Interbank_credit                          0.0
AMT_CREDIT_SUM_DEBT                       0.0
AMT_CREDIT_SUM                            0.0
CNT_CREDIT_PROLONG                        0.0
AMT_CREDIT_MAX_OVERDUE                    0.0
DAYS_ENDDATE_FACT                         0.0
DAYS_CREDIT_ENDDATE                       0.0
CREDIT_DAY_OVERDUE                        0.0
DAYS_CREDIT                               0.0
SK_ID_BUREAU                      

# Bureau Balance

In [27]:
# l_encoder = LabelEncoder()
# X_bu_b_ori['STATUS'] = l_encoder.fit_transform(X_bu_b_ori['STATUS'])
# bub_group = X_bu_b_ori.groupby('SK_ID_BUREAU')

# SK_ID_list = []
# length = []
# one_hots = np.zeros((X_bu_b_ori.SK_ID_BUREAU.nunique(),len(l_encoder.classes_)))
# count = 0
# for key,df in bub_group:
#     SK_ID_list.append(key)
#     length.append(len(df))
#     one_hots[count,df.STATUS.unique()] =1
#     count+=1
#     if count % 10000 == 0:
#         print(count)
        
# df_bu_b = pd.DataFrame({'SK_ID_BUREAU':SK_ID_list,
#                         'STATUS0':one_hots[:,0],
#                         'STATUS1':one_hots[:,1],
#                         'STATUS2':one_hots[:,2],
#                         'STATUS3':one_hots[:,3],
#                         'STATUS4':one_hots[:,4],
#                         'STATUS5':one_hots[:,5],
#                         'STATUS6':one_hots[:,6],
#                         'STATUS7':one_hots[:,7],
#                         'BUREAU_LENGTH':length})
# df_bu_b.to_pickle(ENV.bureau_balance_clean_rnn.value)